In [1]:
import tensorflow as tf
import numpy as np
import gym

In [ ]:
env = gym.make('Cartpole-v0')
env = env.unwrapped
env.seed(1)

In [ ]:
obs_space = 4
act_space = env.action_space.n

max_eps = 300
l_rate = 0.01
gamma = 0.95

In [ ]:
state_in = tf.placeholder(tf.float32, [None, obs_space], name = 'st_in')
action_in = tf.placeholder(tf.float32, [None, act_space], name = 'ac_in')
disc_rewards_in = tf.placeholder(tf.float32, [None, ], name = 'disc_r_in')

#Policy
with tf.name_scope('Policy_stream'):
    
    p_fc1 = tf.layers.dense(state_in, 
                            10, 
                            activation = tf.nn.relu, 
                            kernel_initializer = tf.contrib.layers.xavier_initializer())
    p_fc2 = tf.layers.dense(p_fc1, 
                            act_space, 
                            activation = tf.nn.relu, 
                            kernel_initializer = tf.contrib.layers.xavier_initializer())
    p_fc3 = tf.layers.dense(p_fc2, 
                            act_space, 
                            activation = None, 
                            kernel_initializer = tf.contrib.layers.xavier_initializer())
    
    action_distribution = tf.nn.softmax(p_fc3)
    
    #Policy loss
    neg_log_probs = - tf.reduce_sum(tf.math.multiply(action_in, tf.log(action_distribution)), axis = -1)
    p_loss = tf.reduce_mean(neg_log_probs * disc_rewards_in)

#Value
with tf.name_scope('Value_stream'):
    
    v_fc1 = tf.layers.dense(state_in, 
                            24, 
                            activation = tf.nn.relu, 
                            kernel_initializer = tf.contrib.layers.xavier_initializer())
    value = tf.layers.dense(v_fc1, 
                            1, 
                            activation = None, 
                            kernel_initializer = tf.contrib.layers.xavier_initializer())
    
    #Value loss
    v_loss = tf.reduce_mean(tf.math.square(value - disc_rewards_in))

#Combined loss
alpha = 0.8
loss = p_loss + alpha * v_loss

#Optimizer
opt = tf.train.AdamOptimizer(l_rate).minimize(loss)